In [1]:
pip install pycryptodome pillow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox
from Crypto.Cipher import DES3
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes
import os

# --- Helper Functions ---

def pad(data):
    while len(data) % 8 != 0:
        data += b'\0'
    return data

def get_3des_key(password):
    # Create a 24-byte key from password using SHA256
    hash_obj = SHA256.new(password.encode('utf-8'))
    return DES3.adjust_key_parity(hash_obj.digest()[:24])

def encrypt_file(input_path, output_path, key):
    # Extract file extension
    ext = os.path.splitext(input_path)[1].encode('utf-8')  # e.g., b'.jpg'
    ext = ext.ljust(8, b'\0')  # pad to 8 bytes

    # Read file data
    with open(input_path, 'rb') as f:
        data = f.read()

    data = pad(data)
    cipher = DES3.new(key, DES3.MODE_ECB)
    encrypted_data = cipher.encrypt(data)

    # Save extension + encrypted data
    with open(output_path, 'wb') as f:
        f.write(ext + encrypted_data)

def decrypt_file(input_path, output_dir, key):
    with open(input_path, 'rb') as f:
        content = f.read()

    # First 8 bytes = original extension
    ext = content[:8].rstrip(b'\0').decode('utf-8')
    encrypted_data = content[8:]

    cipher = DES3.new(key, DES3.MODE_ECB)
    decrypted_data = cipher.decrypt(encrypted_data)
    decrypted_data = decrypted_data.rstrip(b'\0')

    # Save file with original extension
    output_path = os.path.join(output_dir, "decrypted_file" + ext)
    with open(output_path, 'wb') as f:
        f.write(decrypted_data)

    return output_path

# --- GUI Functions ---

def select_file():
    file_path = filedialog.askopenfilename(title="Select Image or Encrypted File", filetypes=[("All Files", "*.*")])
    entry_file_path.delete(0, tk.END)
    entry_file_path.insert(0, file_path)

def encrypt_image():
    path = entry_file_path.get()
    password = entry_password.get()
    if not path or not password:
        messagebox.showerror("Error", "Please select a file and enter a password")
        return
    output_path = filedialog.asksaveasfilename(title="Save Encrypted File As", defaultextension=".bin")
    if not output_path:
        return
    try:
        key = get_3des_key(password)
        encrypt_file(path, output_path, key)
        messagebox.showinfo("Success", f"File encrypted successfully!\nSaved at: {output_path}")
    except Exception as e:
        messagebox.showerror("Error", f"Encryption failed: {str(e)}")

def decrypt_image():
    path = entry_file_path.get()
    password = entry_password.get()
    if not path or not password:
        messagebox.showerror("Error", "Please select a file and enter a password")
        return
    output_dir = filedialog.askdirectory(title="Select Folder to Save Decrypted File")
    if not output_dir:
        return
    try:
        key = get_3des_key(password)
        output_path = decrypt_file(path, output_dir, key)
        messagebox.showinfo("Success", f"File decrypted successfully!\nSaved at: {output_path}")
    except Exception as e:
        messagebox.showerror("Error", f"Decryption failed: {str(e)}")

# --- GUI Layout ---

root = tk.Tk()
root.title("3DES Image Encryption Tool")
root.geometry("500x220")
root.resizable(False, False)

tk.Label(root, text="File Path:").pack(pady=5)
entry_file_path = tk.Entry(root, width=50)
entry_file_path.pack(pady=5, padx=10)
tk.Button(root, text="Browse", command=select_file).pack(pady=5)

tk.Label(root, text="Password:").pack(pady=5)
entry_password = tk.Entry(root, width=50, show='*')
entry_password.pack(pady=5, padx=10)

frame_buttons = tk.Frame(root)
frame_buttons.pack(pady=10)

tk.Button(frame_buttons, text="Encrypt", width=15, command=encrypt_image).grid(row=0, column=0, padx=10)
tk.Button(frame_buttons, text="Decrypt", width=15, command=decrypt_image).grid(row=0, column=1, padx=10)

root.mainloop()
